NOTE:
-----

Please run the below cells first before proceeding- you'll need them soon!

In [1]:
%load_ext sql
%sql sqlite:///dataset_1.db

'Connected: @dataset_1.db'

Activity 3-2
------------
Aggregation operators, GROUP BY

Exercise #1
-----------

Consider a set of tables that describe the up-and-coming bagel startup industry; for now let's just look at two tables here, `bagel`, which describes types of bagels made by the different bagel companies:
> * name STRING
> * price FLOAT
> * made_by STRING

And `purchase`:
> * bagel_name STRING
> * franchise STRING
> * date INT
> * quantity INT
> * purchaser_age INT

Where `purchase.bagel_name` references `bagel.name` and `purchase.franchise` references `bagel.made_by`:

In [2]:
%sql SELECT * FROM bagel

 * sqlite:///dataset_1.db
Done.


[('Plain with shmear', 1.99, 'Bobs Bagels'),
 ('Egg with shmear', 2.39, 'Bobs Bagels'),
 ('eBagel Drinkable Bagel', 27.99, 'eBagel'),
 ('eBagel Expansion Pack', 1.99, 'eBagel'),
 ('Plain with shmear', 0.99, 'BAGEL CORP'),
 ('Organic Flax-seed bagel chips', 0.99, 'BAGEL CORP')]

In [3]:
%sql SELECT * FROM purchase

 * sqlite:///dataset_1.db
Done.


[('Plain with shmear', 'Bobs Bagels', 1, 12, 28),
 ('Egg with shmear', 'Bobs Bagels', 2, 6, 47),
 ('Plain with shmear', 'BAGEL CORP', 2, 12, 24),
 ('Plain with shmear', 'BAGEL CORP', 3, 1, 17),
 ('eBagel Expansion Pack', 'eBagel', 1, 137, 5),
 ('Plain with shmear', 'Bobs Bagels', 4, 24, None),
 ('Moonshine', 'BAGEL CORP', 7, 1000, 37)]

Can you write a query to get the _total revenue_ for each bagel type **which had an average purchaser age over 18**?  Type your query below:

In [4]:
%%sql
SELECT b.name, SUM(p.quantity * b.price) AS Revenue
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name
HAVING AVG(p.purchaser_age) > 18;

 * sqlite:///dataset_1.db
Done.


[('Egg with shmear', 14.34), ('Plain with shmear', 84.50999999999999)]

Exercise #2
-----------

Here we'll use a simplified version of the `precipitation_full` table, which just has _daily_ rainfall _in CA only_, and has the following schema:

> * station_id
> * day
> * precipitation

In [5]:
%sql SELECT * FROM precipitation LIMIT 5;

 * sqlite:///dataset_1.db
Done.


[(16102, 1, 10),
 (16102, 4, 10),
 (16102, 24, 30),
 (21201, 1, 0),
 (21201, 20, 10)]

We want to get station_ids which have average precipitations > 75.  Try doing this first as a nested query:

In [6]:
%%sql
SELECT DISTINCT p.station_id
FROM precipitation p 
WHERE (
    SELECT AVG(precipitation) 
    FROM precipitation 
    WHERE station_id = p.station_id) > 75;

 * sqlite:///dataset_1.db
Done.


[(88302,), (250002,), (335701,), (357302,), (488301,)]

Now, try re-writing as a GROUP BY:

In [7]:
%%sql
SELECT station_id
FROM precipitation
GROUP BY station_id
HAVING AVG(precipitation) > 75;

 * sqlite:///dataset_1.db
Done.


[(88302,), (250002,), (335701,), (357302,), (488301,)]

Now time it by using `%time` followed by single-line versions of your queries above (clunky, but will work) to see how they compare!

**Note:** Yes, currently the answers are filled in below for convenience... but you should still try getting them on your own above!

In [8]:
%time %sql SELECT DISTINCT p.station_id FROM precipitation p WHERE (SELECT AVG(precipitation) FROM precipitation WHERE station_id = p.station_id) > 75;

 * sqlite:///dataset_1.db
Done.
CPU times: user 21.8 ms, sys: 1.24 ms, total: 23 ms
Wall time: 21.3 ms


[(88302,), (250002,), (335701,), (357302,), (488301,)]

In [9]:
%time %sql SELECT p.station_id FROM precipitation p GROUP BY p.station_id HAVING AVG(p.precipitation) > 75;

 * sqlite:///dataset_1.db
Done.
CPU times: user 1.9 ms, sys: 794 µs, total: 2.69 ms
Wall time: 1.84 ms


[(88302,), (250002,), (335701,), (357302,), (488301,)]

**An ~ 10-20x difference in execution time!!**